# Graph Convolutional Network - Regression

## Why?

For many problems, switching to graph representation, helps us change the point of view on our problem

By creating as many edges as the surrounding neighbors for each house, we can exploit the message passing layers to 

## Requirements

In [ ]:
def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import scipy 
from scipy.io import arff
from sklearn.model_selection import train_test_split

In [4]:
from IPython.display import Image
Image(url='https://cambridge-intelligence.com/wp-content/uploads/2015/07/mapping-gif.gif')

# https://cambridge-intelligence.com/visualizing-your-geospatial-graph-data-part-1/

## Graph Representation 

In [ ]:
# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt


def describe_dataset(dataset):
    print(f'Dataset: {dataset}:')
    print('======================')
    print(f'Number of graphs: {len(dataset)}')
    print(f'Number of features: {dataset.num_features}')
    print(f'Number of classes: {dataset.num_classes}')
    pass

def describe_graph_obj(graph_obj):
    print(graph_obj)
    print('==============================================================')
    # Gather some statistics about the graph.
    print(f'Number of nodes: {graph_obj.num_nodes}')
    print(f'Number of edges: {graph_obj.num_edges}')
    print(f'Average node degree: {graph_obj.num_edges / graph_obj.num_nodes:.2f}')
    if graph_obj.train_mask is not None:
      print(f'Number of training nodes: {graph_obj.train_mask.sum()}')
      print(f'Training node label rate: {int(graph_obj.train_mask.sum()) / graph_obj.num_nodes:.2f}')
    print(f'Has isolated nodes: {graph_obj.has_isolated_nodes()}')
    print(f'Has self-loops: {graph_obj.has_self_loops()}')
    print(f'Is undirected: {graph_obj.is_undirected()}')
    pass

def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()
    pass

def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()
    pass

## Data Preparation
1. Import Data
2. Perform some transformations or cleaning operations (if not done before)
2. Rescale Data
3. Convert to Tensor
4. Convert Tensor to Graph

In [ ]:
# Import dataset

In [ ]:
# Rescale parameters according

In [ ]:
# Min-Max Scaling
min_max_data

In [ ]:
# Standard Scaling
std_data

scaler = StandardScaler()
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [ ]:
# Train-Test Split
from sklearn.model_selection import train_test_split

# Train-Test Split
X = dataset.drop(labels='Sale_Price', axis=1)
y = dataset['Sale_Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    random_state=42)

In [ ]:
# Convert to Tensors

### Working with graphs

A typical training procedure for a neural network is as follows:
1. Define the neural network that has some learnable parameters (or weights)
2. Iterate over a dataset of inputs
3. Process input through the network
4. Compute the loss (how far is the output from being correct)
5. Propagate gradients back into the network’s parameters
6. Update the weights of the network, typically using a simple update rule

Credits: [NEURAL NETWORKS - PyTorch](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)


In [ ]:
# Estimate edges through k-nn
## Use latitude and longitude

In [ ]:
# Drop latitude and longitude

In [ ]:
# Convert to Graph
import torch_geometric
import torch_cluster
from torch_geometric.data import Data
import torch_geometric.transforms as T

# Transform Dataset to Graph with no edges
def dataset_to_graph(dataset, latitude_index, longitude_index):
  # Get latitude and longitude data
  longitude_index+=1
  lat_long = df.iloc[:, latitude_index:longitude_index]
  # Create a tensor for the edge creation later
  position_tensor = torch.tensor(np.asarray(lat_long), dtype=torch.float32)
  # Create a dataset tensor for the features
  dataset_tensor = torch.tensor(np.asarray(df.drop(lat_long, axis=1)), dtype=torch.float32)
  # Create Graph
  graph = Data(x = dataset_tensor, 
               edge_index = None, 
               edge_attr = None,
               pos = position_tensor, 
               y = None,
               train_mask = torch.ones_like(dataset_tensor, dtype=torch.bool), 
               val_mask = torch.ones_like(dataset_tensor, dtype=torch.bool), 
               test_mask = torch.ones_like(dataset_tensor, dtype=torch.bool))
  return graph

# Cosine only for GPU
def add_edges_knn(graph:Data, k=5, cosine=False):
  edge_creator = T.KNNGraph(k=k, cosine=cosine, force_undirected=True)
  return edge_creator(data=graph)


def add_edges_radius(graph:Data, r=3, max_num_neighbors=10):
  edge_creator = T.RadiusGraph(r=r, max_num_neighbors=max_num_neighbors)
  return edge_creator(data=graph)


In [2]:
from torch_geometric.nn import GCNConv

# Model definition

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

# https://colab.research.google.com/drive/14OvFnAXggxB8vM4e8vSURUp1TaKnovzX?usp=sharing#scrollTo=fmXWs1dKIzD8
# https://docs.dgl.ai/en/0.8.x/guide/training-node.html
# https://github.com/Jiakui/awesome-gcn

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
from torch_geometric.nn import GATConv


class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GATConv(...)  # TODO
        self.conv2 = GATConv(...)  # TODO

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GAT(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(mask):
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
      acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return acc


for epoch in range(1, 201):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

In [ ]:
model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc


for epoch in range(1, 101):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

## Training

## Validation - Parameters Tuning

In [ ]:
import torch.nn.functional as F


class ANN(nn.Module):
  pass

# Define the model
model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)
 
# loss function and optimizer
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)
n_epochs = 100   # number of epochs to run
batch_size = 10  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)
 
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []
 
for epoch in range(n_epochs):
    model.train()
    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(mse=float(loss))
    # evaluate accuracy at end of each epoch
    model.eval()
    y_pred = model(X_test)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    history.append(mse)
    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(model.state_dict())
 
# restore model and return best accuracy
model.load_state_dict(best_weights)
print("MSE: %.2f" % best_mse)
print("RMSE: %.2f" % np.sqrt(best_mse))
plt.plot(history)
plt.show()

## Test

## Diagnostics and Evaluation

Ways to evaluate regression models:
1. Mean/Median of prediction
2. Standard Deviation of prediction
3. Range of prediction
4. Coefficient of Determination (R2)
5. Relative Standard Deviation/Coefficient of Variation (RSD)
6. Relative Squared Error (RSE)
7. Mean Absolute Error (MAE)
8. Relative Absolute Error (RAE)
9. Mean Squared Error (MSE)
10. Root Mean Squared Error on Prediction (RMSE/RMSEP)
11. Normalized Root Mean Squared Error (Norm RMSEP)
12. Relative Root Mean Squared Error (RRMSEP)

Reference:
* [Ways to Evaluate Regression Models](https://towardsdatascience.com/ways-to-evaluate-regression-models-77a3ff45ba70)

In [ ]:
# Evaluation - Predicted vs Actual
## For each epoch display how the following scores change

## R Square and Adjusted R Square

## Mean Square Error(MSE) and Root Mean Square Error(RMSE)

## Mean Absolute Error(MAE)


### Investigating instances

Investigate instances with the most correct predictions and with the most
wrong predictions. Do they have some peculiarities? Any strange feature
distribution?